# 📊 Pipeline de Processamento de Logs do Databricks

Este notebook realiza o processamento dos logs extraídos da API do Databricks, transformando os dados brutos em uma estrutura organizada para análise futura.

In [1]:
# 🚀 1. Instalar dependências
!pip install pandas pyarrow

In [5]:
# 📁 2. Fazer upload do arquivo parquet
from google.colab import files

uploaded = files.upload()  # Escolha o arquivo databricks_logs.parquet

Saving databricks_logs.parquet to databricks_logs (1).parquet


In [7]:
# 📖 3. Ler os dados
import pandas as pd

df = pd.read_parquet('databricks_logs.parquet')
print("Visualização inicial dos dados:")
print(df.head())

Visualização inicial dos dados:
   job_id  run_id           state result_state     start_time       end_time  \
0    1001     201      TERMINATED      SUCCESS  1743501600000  1743503400000   
1    1002     202      TERMINATED       FAILED  1743597000000  1743598200000   
2    1003     203      TERMINATED      SUCCESS  1743691500000  1743693000000   
3    1004     204  INTERNAL_ERROR       FAILED  1743782400000  1743783600000   

  cluster_id  creator_user_name  
0  cluster01  hiago@example.com  
1  cluster02    ana@example.com  
2  cluster03   joao@example.com  
3  cluster04    bia@example.com  


In [10]:
# 🧹 4. Processar os dados

# Converte timestamp
df['start_time'] = pd.to_datetime(df['start_time'], unit='ms')
df['end_time'] = pd.to_datetime(df['end_time'], unit='ms')

# Cria coluna de duração em minutos
df['duration_minutes'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 60

# Cria data da execução
df['data_execucao'] = df['start_time'].dt.date

# Seleciona colunas relevantes (ajuste os nomes conforme seu dataset)
colunas = ['job_id', 'run_id', 'start_time', 'end_time', 'state', 'result_state', 'duration_minutes', 'data_execucao']
df_tratado = df[colunas]

print("\nDados processados:")
print(df_tratado.head())


Dados processados:
   job_id  run_id          start_time            end_time           state  \
0    1001     201 2025-04-01 10:00:00 2025-04-01 10:30:00      TERMINATED   
1    1002     202 2025-04-02 12:30:00 2025-04-02 12:50:00      TERMINATED   
2    1003     203 2025-04-03 14:45:00 2025-04-03 15:10:00      TERMINATED   
3    1004     204 2025-04-04 16:00:00 2025-04-04 16:20:00  INTERNAL_ERROR   

  result_state  duration_minutes data_execucao  
0      SUCCESS              30.0    2025-04-01  
1       FAILED              20.0    2025-04-02  
2      SUCCESS              25.0    2025-04-03  
3       FAILED              20.0    2025-04-04  


In [11]:
# 💾 5. Salvar como novo parquet
df_tratado.to_parquet('logs_processados.parquet', index=False)
print("\nArquivo 'logs_processados.parquet' salvo com sucesso!")


Arquivo 'logs_processados.parquet' salvo com sucesso!


In [12]:
# 📥 6. Baixar o novo arquivo
files.download('logs_processados.parquet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# 📊 7. Análise básica (corrigida)

print("\n📈 Média de duração por *result_state*:")
print(df_tratado.groupby('result_state')['duration_minutes'].mean())

print("\n📅 Quantidade de jobs por dia:")
print(df_tratado.groupby('data_execucao')['job_id'].count())


📈 Média de duração por *result_state*:
result_state
FAILED     20.0
SUCCESS    27.5
Name: duration_minutes, dtype: float64

📅 Quantidade de jobs por dia:
data_execucao
2025-04-01    1
2025-04-02    1
2025-04-03    1
2025-04-04    1
Name: job_id, dtype: int64
